In [1]:
from camping import storage
from camping import ridb_data
import pandas as pd
from geopy.distance import vincenty
import itertools
import numpy as np
import config

In [2]:
request_url = "http://" + config.LAMP_IP + '/ridb_mock.json'
web_data = pd.read_json(request_url)
latlongs = web_data[['FacilityLatitude', 'FacilityLongitude']].as_matrix()

In [6]:
def vincenty_matrix(group):
    group_matrix = pd.DataFrame(index = group.index, columns = group.index)  
    for c in group_matrix.columns:
        lat1 = group.columns.get_loc("FacilityLatitude")
        long1 = group.columns.get_loc("FacilityLongitude")
        lat2 = group.FacilityLatitude[c]
        long2 = group.FacilityLongitude[c]
        group_matrix[c] = group.apply(lambda x: vincenty((x[lat1], x[long1]), (lat2,long2)).miles, axis=1)
    return group_matrix.as_matrix()

In [7]:
distance_matrix = vincenty_matrix(web_data)

In [49]:
matches = np.where((np.triu(distance_matrix) < 1) & (np.triu(distance_matrix) > 0))

In [50]:
matches

(array([ 3, 15]), array([ 5, 16]))

In [57]:
for pair in zip(matches[0],matches[1]):
     print(pair)

(3, 5)
(15, 16)


In [46]:
web_data_copy = web_data

In [68]:
for pair in zip(matches[0],matches[1]):
    pair_sub = web_data.loc[np.array(pair)]
    print(pair_sub.LastUpdatedDate)
    least_recent = pair_sub[pair_sub.LastUpdatedDate != max(pair_sub.LastUpdatedDate)]
    web_data_copy = web_data_copy.drop(least_recent.index)

3    2016-05-09
5    2015-10-15
Name: LastUpdatedDate, dtype: object
15    2016-05-09
16    2015-12-03
Name: LastUpdatedDate, dtype: object


In [70]:
web_data_copy.index

Int64Index([0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], dtype='int64')

In [17]:
pd.DataFrame(distance_matrix)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.000000,100.900530,46.163737,12.976290,64.103978,12.175875,19.562382,17.211835,25.018834,18.612140,19.438193,7733.425088,6.979180,163.073510,21.303609,19.983981,19.982539
1,100.900530,0.000000,83.734564,100.938215,78.025573,100.569274,89.491834,93.540246,81.788054,82.353279,81.909957,7682.877439,93.943853,76.959602,82.420492,111.547075,111.545795
2,46.163737,83.734564,0.000000,56.930092,18.479713,56.074328,26.765804,55.780765,52.768527,37.364158,34.974674,7753.217640,43.073363,158.121638,48.073295,66.038563,66.037081
3,12.976290,100.938215,56.930092,0.000000,74.142274,0.865389,30.285620,8.011761,19.666662,22.837837,24.750991,7721.789836,13.856812,158.263553,18.552143,11.104309,11.102843
4,64.103978,78.025573,18.479713,74.142274,0.000000,73.304475,44.545719,72.043606,67.172547,52.990975,50.657334,7756.047360,60.382097,154.601457,62.959324,83.763191,83.761685
5,12.175875,100.569274,56.074328,0.865389,73.304475,0.000000,29.423587,8.001545,19.480829,22.116835,24.001729,7722.370359,13.001124,158.235513,18.153645,11.717293,11.715805
6,19.562382,89.491834,26.765804,30.285620,44.545719,29.423587,0.000000,30.126416,30.394952,14.742527,12.888113,7740.053339,16.472163,157.473200,25.293603,39.299137,39.297646
7,17.211835,93.540246,55.780765,8.011761,72.043606,8.001545,30.126416,0.000000,11.852961,19.152904,21.426614,7716.213254,14.480998,150.255771,11.772738,18.006988,18.005711
8,25.018834,81.788054,52.768527,19.666662,67.172547,19.480829,30.394952,11.852961,0.000000,15.987911,18.262591,7710.712945,19.366320,139.056452,5.101925,29.810716,29.809480
9,18.612140,82.353279,37.364158,22.837837,52.990975,22.116835,14.742527,19.152904,15.987911,0.000000,2.389617,7725.400631,11.762580,146.369798,10.976231,33.735543,33.734024
